# Titanic: Machine learning from disaster

## Kaggle getting start prediction competition

# Descrição&nbsp;do&nbsp;trabalho

Neste _notebook_ vamos fazer resolver o _challenge_ "_Titanic: Machine learning from disaster_" do [kaggle](https://www.kaggle.com/c/titanic), para exemplificar o uso do python pandas aplicado a um problema de _machine learning_.

Resumo do notebook:

1. [Descrição do dataset](#Descrição&nbsp;do&nbsp;dataset)
<br>Informação detalhada sobre o _dataset_.<br><br>
2. [Importar bibliotecas e dataset](#Importar&nbsp;bibliotecas&nbsp;e&nbsp;dataset)
<br> Importar as bibliotecas necessárias e importar o _dataset_ do kaggle.<br><br>
3. [Análise exploratória do dataset](# Análise&nbsp;exploratória&nbsp;do&nbsp;dataset) 
<br> Pequeno conjunto de operações para ter uma ideia das características do _dataset_. <br><br>
4. [Modelo e aprendizagem supervisionada](#Modelo&nbsp;e&nbsp;aprendizagem&nbsp;supervisionada) 
<br>Aplicação de um conjunto de modelos da aprendizagem supervisionada no _dataset_ e respectiva avaliação.<br><br>
5. [Output para submissão](#Output&nbsp;para&nbsp;submissão) 
<br>Preparação do ficheiro para submissão na competição do [kaggle](https://www.kaggle.com/c/titanic). Serão feitas duas submissões:<br>
 - totalmente aleatória <br>
 - usando o melhor modelo treinado no ponto anterior.<br>



---
# Descrição&nbsp;do&nbsp;dataset

## Overview

The data has been split into two groups:

- training set (train.csv)
- test set (test.csv)



The training set should be used to build your machine learning models. For the training set, we provide the outcome (also known as the “ground truth”) for each passenger. Your model will be based on “features” like passengers’ gender and class. You can also use feature engineering to create new features.

The test set should be used to see how well your model performs on unseen data. For the test set, we do not provide the ground truth for each passenger. It is your job to predict these outcomes. For each passenger in the test set, use the model you trained to predict whether or not they survived the sinking of the Titanic.

We also include gender_submission.csv, a set of predictions that assume all and only female passengers survive, as an example of what a submission file should look like.

## Data Dictionary


| Variable | Definition | Key |
|:-|:-|:-|
|survival |	Survival |	0 = No, 1 = Yes |
|pclass |	Ticket class | 	1 = 1st, 2 = 2nd, 3 = 3rd |
|sex |	Sex | | 
|Age |	Age in years 	| | 
|sibsp |	# of siblings / spouses aboard the Titanic 	 | |
|parch |	# of parents / children aboard the Titanic 	 | |
|ticket |	Ticket number 	 | |
|fare |	Passenger fare 	 | |
|cabin |	Cabin number 	 | |
|embarked |	Port of Embarkation |	C = Cherbourg, Q = Queenstown, S = Southampton|

## Variable Notes

**pclass**: A proxy for socio-economic status (SES)<br>
1st = Upper <br>
2nd = Middle <br>
3rd = Lower <br>

**age**: Age is fractional if less than 1. If the age is estimated, is it in the form of xx.5

**sibsp**: The dataset defines family relations in this way...<br>
Sibling = brother, sister, stepbrother, stepsister<br>
Spouse = husband, wife (mistresses and fiancés were ignored)

**parch**: The dataset defines family relations in this way...<br>
Parent = mother, father<br>
Child = daughter, son, stepdaughter, stepson<br>
Some children travelled only with a nanny, therefore parch=0 for them.

[Topo](#Descrição&nbsp;do&nbsp;trabalho)

---

# Importar&nbsp;bibliotecas&nbsp;e&nbsp;dataset


In [ ]:
%pylab inline
import pandas as pd
import seaborn as sns

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train.head()

In [ ]:
train.info()

## Construir dois datasets:

- `full`: constituído pelo conjunto _training set_ e _test set_
- `titanic`: constituído pelo _training set_

In [ ]:
full = train.append(test, ignore_index=True)
titanic = full[:891]

In [ ]:
full.info()

In [ ]:
titanic.info()

In [ ]:
full.head()

In [ ]:
titanic.describe()

[Topo](#Descrição&nbsp;do&nbsp;trabalho)

---

# Análise&nbsp;exploratória&nbsp;do&nbsp;dataset

## Correlação entre as variáveis

O gráfico seguinte mostra a matriz de correlação como um _heatmap_ para ilustrar a dependência das diferentes variáveis

In [ ]:
figure(figsize =( 8 , 7 ))
sns.heatmap(titanic.corr(), annot = True)

## Distribuição das variáveis

A função seguinte permite ilustrar a distribuição de uma variável.

### Taxa de sobrevivência em função da idade e sexo

In [ ]:
var = 'Age'
target = 'Survived'
row = 'Sex'
facet = sns.FacetGrid(titanic, hue=target, aspect=3, row=row)
facet.map(sns.kdeplot, var, shade=True)
facet.set(xlim=( 0 , titanic[var].max()))
facet.add_legend();

### Taxa de sobrevivência em função do preço do bilhete

In [ ]:
var = 'Fare'
target = 'Survived'
row = 'Sex'
facet = sns.FacetGrid(titanic, hue=target, aspect=3, row=row)
facet.map(sns.kdeplot, var, shade=True)
facet.set(xlim=( 0 , titanic[var].max()))
facet.add_legend();

### Taxa de sobrevivência em função da porta de embarque

In [ ]:
cat = 'Embarked'
target = 'Survived'
facet = sns.FacetGrid( titanic )
facet.map( sns.barplot , cat , target )
facet.add_legend();

### Taxa de sobrevivência em função da classe do bilhete

In [ ]:
cat = 'Pclass'
target = 'Survived'
facet = sns.FacetGrid( titanic )
facet.map( sns.barplot , cat , target )
facet.add_legend();

### Taxa de sobrevivência em função do número de familiares directos (siblings/spouses)

In [ ]:
cat = 'SibSp'
target = 'Survived'
facet = sns.FacetGrid( titanic )
facet.map( sns.barplot , cat , target )
facet.add_legend();

### Taxa de sobrevivência em função do número de famíliares de geração anterior e seguinte (parents/children)

In [ ]:
cat = 'Parch'
target = 'Survived'
facet = sns.FacetGrid( titanic )
facet.map( sns.barplot , cat , target )
facet.add_legend();

[Topo](#Descrição&nbsp;do&nbsp;trabalho)

---

# Transformar&nbsp;o&nbsp;dataset

Antes de aplicar algoritmos de _machine learning_, serão realizadas as seguintes transformações do dataset:

- [Transformar categorias em números](#Transformar&nbsp;categorias&nbsp;em&nbsp;números)
- [Preencher os missing values](#Preencher&nbsp;os&nbsp;missing&nbsp;values)
- [Criar o dataset final para o modelo](#Criar&nbsp;o&nbsp;dataset&nbsp;final&nbsp;para&nbsp;o&nbsp;modelo)


## Transformar&nbsp;categorias&nbsp;em&nbsp;números

In [ ]:
# Transformar Sex em valores binários: 0 ou 1
sex = pd.Series( np.where( full.Sex == 'male' , 1 , 0 ) , name = 'Sex' )
sex.head()

In [ ]:
# Criar uma nova variável numérica única para cada sítio de embarque
embarked = pd.get_dummies( full.Embarked , prefix='Embarked' )
embarked.head()

In [ ]:
# Criar uma nova variável numérica única para cada classe do bilhete
pclass = pd.get_dummies( full.Pclass , prefix='Pclass' )
pclass.head()

## Preencher&nbsp;os&nbsp;missing&nbsp;values

In [ ]:
# Novo dataset
filledin = pd.DataFrame()

# Preencher os missing values da idade com a idade média dos passageiros
filledin[ 'Age' ] = full.Age.fillna( full.Age.mean() )

# Preencher o valor da tarifa do bilhete com o preço médio dos bilhetes
filledin[ 'Fare' ] = full.Fare.fillna( full.Fare.mean() )

filledin.head()

## Criar&nbsp;o&nbsp;dataset&nbsp;final&nbsp;para&nbsp;o&nbsp;modelo

In [ ]:
full_model = pd.concat( [ filledin , embarked , pclass, sex ] , axis=1 )
full_model.head()

[Topo](#Descrição&nbsp;do&nbsp;trabalho)

---

# Modelo&nbsp;e&nbsp;aprendizagem&nbsp;supervisionada


## Importar os modelos e definição dos datasets para os modelos

In [1]:
# Funções do módulo sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

### Conjunto de treino e teste obtido a partir do dataset final:  `full_model`

In [ ]:
# Criar os datasets para treinar, validar e testar os modelos
train_valid_X = full_model[ 0:891 ]
train_valid_y = titanic.Survived
test_X = full_model[ 891: ]

### Separação em conjunto de teste e treino


In [ ]:
train_X , valid_X , train_y , valid_y = train_test_split( train_valid_X , train_valid_y,  train_size = .7)

In [ ]:
print('Dimensões dos datasets:')
print ('full_model:', full_model.shape)
print('train_X:', train_X.shape)
print('valid_X:', valid_X.shape)
print('train_y:', train_y.shape)
print('valid_y:', valid_y.shape)
print('test_X:', test_X.shape)

## K-nears neighbors

In [ ]:
model = KNeighborsClassifier(n_neighbors = 2)
model.fit( train_X , train_y )

In [ ]:
print("Score do conjunto de treino:", model.score( train_X , train_y ))
print("Score do conjunto de teste:", model.score( valid_X , valid_y ))

## Logistic Regression

In [ ]:
model = LogisticRegression(C=0.1)
model.fit( train_X , train_y )

In [ ]:
print("Score do conjunto de treino:", model.score( train_X , train_y ))
print("Score do conjunto de teste:", model.score( valid_X , valid_y ))

## SVM linear

In [ ]:
model = LinearSVC(C=0.1)
model.fit( train_X , train_y)

In [ ]:
print("Score do conjunto de treino:", model.score( train_X , train_y ))
print("Score do conjunto de teste:", model.score( valid_X , valid_y ))

## SVM

In [ ]:
model = SVC(C=10)
model.fit( train_X , train_y)

In [ ]:
print("Score do conjunto de treino:", model.score( train_X , train_y ))
print("Score do conjunto de teste:", model.score( valid_X , valid_y ))

## Decision trees

In [ ]:
model = DecisionTreeClassifier( random_state = 99 )
model.fit( train_X , train_y )


In [ ]:
print("Score do conjunto de treino:", model.score( train_X , train_y ))
print("Score do conjunto de teste:", model.score( valid_X , valid_y ))

## Random Forest

In [ ]:
model = RandomForestClassifier(n_estimators=100)
model.fit( train_X , train_y )
print("Score do conjunto de treino:", model.score( train_X , train_y ))
print("Score do conjunto de teste:", model.score( valid_X , valid_y ))

### Importância das features no random forest

In [ ]:
imp = pd.DataFrame( 
        model.feature_importances_  , 
        columns = [ 'Importance' ] , 
        index = train_X.columns 
    )
imp = imp.sort_values( [ 'Importance' ] , ascending = True )
imp.plot( kind = 'barh' )

[Topo](#Descrição&nbsp;do&nbsp;trabalho)

---

# Output&nbsp;para&nbsp;submissão

## Submissão aleatória

In [ ]:
test_Y = np.random.randint(2, size=len(full[891:]))
passenger_id = full[891:].PassengerId
test = pd.DataFrame( { 'PassengerId': passenger_id , 'Survived': test_Y.astype(int) } )
test.shape
test.head()

In [ ]:
test.to_csv( 'titanic_pred_random.csv' , index = False )

## Submissão usando o nosso melhor modelo

In [ ]:
model = RandomForestClassifier(n_estimators=100)
model.fit( train_X , train_y )
test_Y = model.predict( test_X )
passenger_id = full[891:].PassengerId
test = pd.DataFrame( { 'PassengerId': passenger_id , 'Survived': test_Y.astype(int) } )
test.shape
test.head()

In [ ]:
test.to_csv( 'titanic_pred.csv' , index = False )